<a href="https://colab.research.google.com/github/GitSid-glitch/ML_Colab/blob/main/Copy_of_Translation_Question_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Libraries

* **Import Libraries:** This cell imports the necessary libraries for data processing and neural network building.

**Hints:**
- Import `pandas` for data manipulation
- Import `torch` and `torch.nn` for neural networks
- Import `random` and `time` for shuffling and timing

**What each function does:**
| Function | Description |
|----------|-------------|
| `random.seed(42)` | Sets Python's random number generator to a fixed state for reproducible results |
| `torch.manual_seed(42)` | Sets PyTorch's random seed so neural network weights initialize the same way every run |

**Documentation:**
- [pandas](https://pandas.pydata.org/docs/) - Data analysis library
- [torch](https://pytorch.org/docs/stable/torch.html) - PyTorch tensor operations
- [torch.nn](https://pytorch.org/docs/stable/nn.html) - Neural network building blocks
- [random.seed](https://docs.python.org/3/library/random.html#random.seed) - Initialize random generator
- [torch.manual_seed](https://pytorch.org/docs/stable/generated/torch.manual_seed.html) - Set PyTorch random seed

In [2]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Dataset_English_Hindi.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "preetviradiya/english-hindi-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/tmp/ipython-input-1790220126.py:10: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'english-hindi-dataset' dataset.
First 5 records:   English    Hindi
0   Help!    बचाओ!
1   Jump.    उछलो.
2   Jump.    कूदो.
3   Jump.   छलांग.
4  Hello!  नमस्ते।


# Dataset URL
https://www.kaggle.com/datasets/preetviradiya/english-hindi-dataset

In [3]:
import pandas as pd
import torch  # Hint: PyTorch deep learning library
import torch.nn as nn
import random
import time

# Set seeds for reproducibility
random.seed(42)
torch.manual_seed(42)  # Hint: Function to set PyTorch's random seed

print("Libraries loaded!")

Libraries loaded!


## 2. Load Data

* **Load Dataset:** This cell loads the English-Hindi translation dataset from a CSV file and displays sample translations.

**Hints:**
- Use `pd.read_csv()` to load CSV files
- Use `len(df)` to get number of rows
- Access columns with `df['column'][index]`

**What each function does:**
| Function | Description |
|----------|-------------|
| `pd.read_csv('file.csv')` | Reads a CSV file and returns a DataFrame (table) with rows and columns |
| `len(df)` | Returns the total number of rows in the DataFrame |
| `df['column'][i]` | Accesses the value at row `i` of the specified column |

**Documentation:**
- [pd.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) - Read CSV into DataFrame
- [DataFrame indexing](https://pandas.pydata.org/docs/user_guide/indexing.html) - Access DataFrame elements

In [4]:
# Load dataset
# df = pd.read_csv('Dataset_English_Hindi.csv')  # Hint: Function to read CSV files
print(f"Total sentences: {len(df)}")
print("\nFirst 5 examples:")
for i in range(5):
    print(f"  {df['English'][i]:18} → {df['Hindi'][i]}")

Total sentences: 130476

First 5 examples:
  Help!              → बचाओ!
  Jump.              → उछलो.
  Jump.              → कूदो.
  Jump.              → छलांग.
  Hello!             → नमस्ते।


## 3. Prepare Data Subset

* **Filter Data:** This cell filters the dataset to select short sentences and removes rows with missing data.

**Hints:**
- Use `df[condition]` for boolean indexing
- Use `.str.len()` to get string lengths
- Use `.head(n)` to select first n rows
- Use `.dropna()` to remove NaN values
- Use `.reset_index(drop=True)` to reset indices

**What each function does:**
| Function | Description |
|----------|-------------|
| `df['col'].str.len()` | Returns the character length of each string in the column |
| `df[condition]` | Filters rows where condition is True (boolean indexing) |
| `df.head(n)` | Returns first n rows of the DataFrame |
| `df.dropna(subset=['cols'])` | Removes rows that have NaN (missing) values in specified columns |
| `df.reset_index(drop=True)` | Resets row indices to 0,1,2,... and drops old index |

**Documentation:**
- [DataFrame.str.len](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.len.html) - String length
- [DataFrame.head](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) - First n rows
- [DataFrame.dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) - Remove missing values
- [DataFrame.reset_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) - Reset row numbers

In [5]:
# Use 500 short sentences for faster training
df_small = df[df['English'].str.len() < 30].head(1000).reset_index(drop=True)  # Hint: Get first n rows

# Remove rows with missing values
df_small = df_small.dropna(subset=['English', 'Hindi']).reset_index(drop=True)  # Hint: Remove NaN rows

print(f"Selected {len(df_small)} sentences for training")

Selected 1000 sentences for training


## 5. Helper Function: Text to Numbers

* **Convert Text to Indices:** This cell defines a function to convert sentences into sequences of numbers.

**Hints:**
- Use list comprehension with `dict.get()` for word lookup
- Append `<EOS>` token (index 2) at the end
- Use `torch.tensor()` to create PyTorch tensor

**What each function does:**
| Function | Description |
|----------|-------------|
| `word2idx.get(word, 3)` | Looks up word's index, returns 3 (UNK) if word not found |
| `list.append(item)` | Adds item to end of list |
| `torch.tensor(data, dtype)` | Creates a PyTorch tensor from Python list |
| `tensor.tolist()` | Converts PyTorch tensor back to Python list |

**Documentation:**
- [dict.get](https://docs.python.org/3/library/stdtypes.html#dict.get) - Safe dictionary lookup
- [torch.tensor](https://pytorch.org/docs/stable/generated/torch.tensor.html) - Create tensor
- [Tensor.tolist](https://pytorch.org/docs/stable/generated/torch.Tensor.tolist.html) - Tensor to list

In [6]:
def build_vocab(sentences, min_freq=2):
    """Build word to index mapping"""
    word2idx = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}

    # Count words
    word_counts = {}
    for sentence in sentences:
        # Convert to string and skip if empty or NaN
        sentence_str = str(sentence) if pd.notna(sentence) else ""
        if sentence_str and sentence_str != 'nan':
            for word in sentence_str.lower().split():  # Hint: Convert to lowercase
                word_counts[word] = word_counts.get(word, 0) + 1  # Hint: Safe dict lookup with default

    # Add frequent words
    idx = 4
    for word, count in word_counts.items():  # Hint: Get key-value pairs from dict
        if count >= min_freq:
            word2idx[word] = idx
            idx += 1

    idx2word = {v: k for k, v in word2idx.items()}
    return word2idx, idx2word

# Build vocabularies
eng_word2idx, eng_idx2word = build_vocab(df_small['English'].tolist())
hin_word2idx, hin_idx2word = build_vocab(df_small['Hindi'].tolist())

print(f"English vocabulary: {len(eng_word2idx)} words")
print(f"Hindi vocabulary: {len(hin_word2idx)} words")

English vocabulary: 489 words
Hindi vocabulary: 535 words


## 4. Build Vocabulary

* **Create Vocabulary:** This cell creates word-to-index mappings for both English and Hindi.

**Hints:**
- Initialize with special tokens: `<PAD>=0, <SOS>=1, <EOS>=2, <UNK>=3`
- Use `dict.get(key, default)` for safe dictionary access
- Use `pd.notna()` to check for NaN values
- Use `str.lower().split()` to tokenize sentences

**What each function does:**
| Function | Description |
|----------|-------------|
| `dict.get(key, default)` | Returns value for key if exists, otherwise returns default value |
| `pd.notna(value)` | Returns True if value is NOT missing (not NaN) |
| `str.lower()` | Converts string to lowercase ("Hello" → "hello") |
| `str.split()` | Splits string into list of words by whitespace |
| `dict.items()` | Returns all key-value pairs from dictionary |

**Documentation:**
- [dict.get](https://docs.python.org/3/library/stdtypes.html#dict.get) - Safe dictionary access
- [pd.notna](https://pandas.pydata.org/docs/reference/api/pandas.notna.html) - Check for non-null
- [str.lower](https://docs.python.org/3/library/stdtypes.html#str.lower) - Lowercase conversion
- [str.split](https://docs.python.org/3/library/stdtypes.html#str.split) - Split into words

In [7]:
def sentence_to_indices(sentence, word2idx):
    """Convert sentence to list of indices"""
    indices = [word2idx.get(word.lower(), 3) for word in sentence.split()]  # Hint: Safe dict lookup
    indices.append(2)  # Hint: Add EOS token to end of list
    return torch.tensor(indices, dtype=torch.long)

# Example
example = df_small['English'][0]
indices = sentence_to_indices(example, eng_word2idx)
print(f"'{example}' → {indices.tolist()}")

'Help!' → [3, 2]


## 6. Build Encoder

* **Define Encoder:** This cell creates the Encoder neural network that reads English sentences and produces a context vector.

**Hints:**
- Inherit from `nn.Module`
- Use `nn.Embedding(vocab_size, embed_size)` for word embeddings
- Use `nn.GRU(embed_size, hidden_size, batch_first=True)` for sequence processing
- Return the final hidden state as context

**What each function does:**
| Function | Description |
|----------|-------------|
| `nn.Module` | Base class for all neural networks - provides training/saving functionality |
| `super().__init__()` | Initializes the parent nn.Module class |
| `nn.Embedding(vocab_size, embed_size)` | Creates a lookup table that converts word indices to dense vectors |
| `nn.GRU(input, hidden, batch_first)` | Gated Recurrent Unit - processes sequences and remembers context |
| `self.embedding(x)` | Converts input indices to embedding vectors |
| `self.gru(embedded)` | Processes sequence, returns (outputs, final_hidden_state) |

**Documentation:**
- [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) - Base neural network class
- [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) - Word to vector lookup
- [nn.GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html) - Recurrent layer for sequences

In [8]:
class Encoder(nn.Module):  # Hint: Base class for PyTorch neural networks
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # Hint: Converts word indices to vectors
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        _, hidden = self.gru(embedded)
        return hidden

print("Encoder defined")

Encoder defined


## 7. Build Decoder

* **Define Decoder:** This cell creates the Decoder neural network that generates Hindi words one at a time using the context.

**Hints:**
- Use `nn.Embedding` for Hindi word embeddings
- Use `nn.GRU` to process with hidden state
- Use `nn.Linear(hidden_size, vocab_size)` to predict next word
- Use `tensor.squeeze(1)` to remove batch dimension

**What each function does:**
| Function | Description |
|----------|-------------|
| `nn.Embedding(vocab_size, embed_size)` | Converts Hindi word indices to dense vectors |
| `nn.GRU(embed_size, hidden_size)` | Processes current word + hidden state → next hidden state |
| `nn.Linear(hidden_size, vocab_size)` | Fully connected layer that predicts probability for each word |
| `output.squeeze(1)` | Removes dimension of size 1 (e.g., shape [1,256] → [256]) |

**Documentation:**
- [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) - Word embeddings
- [nn.GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html) - Recurrent layer
- [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) - Fully connected layer
- [Tensor.squeeze](https://pytorch.org/docs/stable/generated/torch.squeeze.html) - Remove size-1 dimensions

In [9]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)  # Hint: Recurrent layer type
        self.fc = nn.Linear(hidden_size, vocab_size)  # Hint: Fully connected layer

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.gru(embedded, hidden)
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden

print("Decoder defined")

Decoder defined


## 8. Prepare Training Data

* **Convert to Tensors:** This cell converts all sentence pairs from text to tensor format for training.

**Hints:**
- Loop through all rows with `range(len(df))`
- Use `sentence_to_indices()` for both English and Hindi
- Store pairs as tuples in a list

**What each function does:**
| Function | Description |
|----------|-------------|
| `range(len(df))` | Creates sequence 0,1,2,...,n-1 for iterating through rows |
| `sentence_to_indices(text, vocab)` | Our custom function - converts sentence to tensor of indices |
| `list.append((a, b))` | Adds a tuple (English tensor, Hindi tensor) to the list |

**Documentation:**
- [range](https://docs.python.org/3/library/functions.html#func-range) - Generate number sequence
- [list.append](https://docs.python.org/3/tutorial/datastructures.html) - Add item to list

In [10]:
# Convert all pairs to tensors
training_pairs = []
for idx in range(len(df_small)):  # Hint: Get number of rows
    eng_tensor = sentence_to_indices(df_small['English'][idx], eng_word2idx)
    hin_tensor = sentence_to_indices(df_small['Hindi'][idx], hin_word2idx)
    training_pairs.append(    (eng_tensor, hin_tensor))  # Hint: Add item to list

print(f"{len(training_pairs)} training pairs ready")

1000 training pairs ready


## 9. Create Models

* **Initialize Models:** This cell creates encoder/decoder instances and sets up loss function and optimizers.

**Hints:**
- Set `EMBED_SIZE=128` and `HIDDEN_SIZE=256`
- Use `nn.CrossEntropyLoss()` for classification loss
- Use `torch.optim.Adam(model.parameters(), lr=0.001)` for optimization

**What each function does:**
| Function | Description |
|----------|-------------|
| `Encoder(vocab, embed, hidden)` | Creates encoder instance with specified sizes |
| `Decoder(vocab, embed, hidden)` | Creates decoder instance with specified sizes |
| `nn.CrossEntropyLoss()` | Loss function that measures how wrong predictions are (lower = better) |
| `torch.optim.Adam(params, lr)` | Optimizer that updates weights to minimize loss; lr = learning rate |
| `model.parameters()` | Returns all trainable weights in the model |

**Documentation:**
- [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) - Classification loss
- [torch.optim.Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html) - Adam optimizer
- [Module.parameters](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters) - Get model weights

In [11]:
# Model parameters
EMBED_SIZE = 128
HIDDEN_SIZE = 256

# Create models
encoder = Encoder(len(eng_word2idx), EMBED_SIZE, HIDDEN_SIZE)
decoder = Decoder(len(hin_word2idx), EMBED_SIZE, HIDDEN_SIZE)

# Setup training
criterion = nn.CrossEntropyLoss()  # Hint: Loss function for classification
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)  # Hint: Adaptive optimizer
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)  # Hint: Same optimizer

print(f"Models ready!")
print(f"  Embedding: {EMBED_SIZE}, Hidden: {HIDDEN_SIZE}")

Models ready!
  Embedding: 128, Hidden: 256


## 10. Training Function

* **Define Training Step:** This cell defines the function to train on one English-Hindi sentence pair.

**Hints:**
- Call `optimizer.zero_grad()` before forward pass
- Use `tensor.unsqueeze(0)` to add batch dimension
- Use `loss.backward()` to compute gradients
- Call `optimizer.step()` to update weights

**What each function does:**
| Function | Description |
|----------|-------------|
| `optimizer.zero_grad()` | Clears old gradients - must do before each training step |
| `tensor.unsqueeze(0)` | Adds a dimension at position 0 (e.g., [5] → [1,5] for batch) |
| `encoder(input)` | Runs encoder forward pass, returns context vector |
| `decoder(input, hidden)` | Runs decoder forward pass, returns (prediction, new_hidden) |
| `criterion(pred, target)` | Calculates loss between prediction and actual target |
| `loss.backward()` | Computes gradients via backpropagation |
| `optimizer.step()` | Updates model weights using computed gradients |
| `loss.item()` | Extracts Python number from single-element tensor |

**Documentation:**
- [Optimizer.zero_grad](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html) - Clear gradients
- [Tensor.unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html) - Add dimension
- [Tensor.backward](https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html) - Backpropagation
- [Optimizer.step](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html) - Update weights

In [12]:
def train_one_pair(eng_tensor, hin_tensor):
    """Train on one sentence pair"""
    enc_optimizer.zero_grad()  # Hint: Clear old gradients
    dec_optimizer.zero_grad()  # Hint: Clear old gradients

    # Encode
    context = encoder(eng_tensor.unsqueeze(0))

    # Decode word by word
    loss = 0
    hidden = context
    for i in range(hin_tensor.size(0)):
        dec_input = torch.tensor([[1]]) if i == 0 else hin_tensor[i-1].unsqueeze(0).unsqueeze(0)
        output, hidden = decoder(dec_input, hidden)
        loss += criterion(output, hin_tensor[i].unsqueeze(0))

    # Update weights
    loss.backward()  # Hint: Compute gradients
    enc_optimizer.step()  # Hint: Update weights
    dec_optimizer.step()  # Hint: Update weights

    return loss.item() / hin_tensor.size(0)

print("Training function ready")

Training function ready


## 11. Train the Model

* **Training Loop:** This cell runs the training loop for 100 epochs (~5-10 minutes).

**Hints:**
- Use `random.shuffle()` to randomize order each epoch
- Use `time.time()` to track elapsed time
- Print progress every 10 epochs with `epoch % 10 == 0`

**What each function does:**
| Function | Description |
|----------|-------------|
| `range(1, n+1)` | Creates sequence 1,2,3,...,n for epoch counting |
| `random.shuffle(list)` | Randomly reorders list in-place (prevents memorizing order) |
| `time.time()` | Returns current time in seconds (for measuring duration) |
| `epoch % 10 == 0` | True every 10 epochs (modulo operator for progress printing) |

**Documentation:**
- [random.shuffle](https://docs.python.org/3/library/random.html#random.shuffle) - Randomize list order
- [time.time](https://docs.python.org/3/library/time.html#time.time) - Current timestamp

In [13]:
NUM_EPOCHS = 100
print("Training started...")
print("=" * 60)

start_time = time.time()  # Hint: Get current timestamp

for epoch in range(1, NUM_EPOCHS + 1):
    random.shuffle(training_pairs)  # Hint: Randomize list order
    total_loss = 0

    for eng_tensor, hin_tensor in training_pairs:
        loss = train_one_pair(eng_tensor, hin_tensor)
        total_loss += loss

    if epoch % 10 == 0:
        avg_loss = total_loss / len(training_pairs)
        print(f"Epoch {epoch:2d}/{NUM_EPOCHS} | Loss: {avg_loss:.3f} | Time: {time.time()-start_time:.1f}s")

print("=" * 60)
print("Training complete!")

Training started...
Epoch 10/100 | Loss: 0.212 | Time: 148.4s
Epoch 20/100 | Loss: 0.165 | Time: 290.8s
Epoch 30/100 | Loss: 0.148 | Time: 432.0s
Epoch 40/100 | Loss: 0.135 | Time: 580.2s
Epoch 50/100 | Loss: 0.129 | Time: 724.1s
Epoch 60/100 | Loss: 0.124 | Time: 874.0s
Epoch 70/100 | Loss: 0.123 | Time: 1028.7s
Epoch 80/100 | Loss: 0.120 | Time: 1200.8s
Epoch 90/100 | Loss: 0.120 | Time: 1354.8s
Epoch 100/100 | Loss: 0.116 | Time: 1507.6s
Training complete!


## 12. Translation Function

* **Define Inference:** This cell defines the function to translate English sentences to Hindi.

**Hints:**
- Use `torch.no_grad()` to disable gradient computation
- Start decoder with `<SOS>` token (index 1)
- Use `output.argmax(1)` to get predicted word index
- Stop when `<EOS>` token (index 2) is generated
- Use `' '.join()` to combine words

**What each function does:**
| Function | Description |
|----------|-------------|
| `torch.no_grad()` | Context manager that disables gradient tracking (faster, less memory) |
| `encoder(input)` | Encodes English sentence into context vector |
| `torch.tensor([[1]])` | Creates tensor with value 1 (SOS token) as decoder starting input |
| `output.argmax(1)` | Returns index of highest probability word (the prediction) |
| `tensor.item()` | Converts single-element tensor to Python int |
| `dict.get(key, default)` | Looks up word for index, returns UNK if not found |
| `' '.join(list)` | Combines list of words into single string with spaces |

**Documentation:**
- [torch.no_grad](https://pytorch.org/docs/stable/generated/torch.no_grad.html) - Disable gradients
- [Tensor.argmax](https://pytorch.org/docs/stable/generated/torch.argmax.html) - Index of max value
- [Tensor.item](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html) - Tensor to Python number
- [str.join](https://docs.python.org/3/library/stdtypes.html#str.join) - Join strings

In [16]:
def translate(sentence, max_len=20):
    """Translate English to Hindi"""
    eng_tensor = sentence_to_indices(sentence, eng_word2idx)

    with torch.no_grad():  # Hint: Disable gradient computation
        context = encoder(eng_tensor.unsqueeze(0))

    hindi_words = []
    hidden = context
    dec_input = torch.tensor([[1]])  # <SOS>

    for _ in range(max_len):
        with torch.no_grad():  # Hint: Same as above
            output, hidden = decoder(dec_input, hidden)

        predicted_idx = output.argmax(1).item()  # Hint: Get index of max value
        if predicted_idx == 2:  # <EOS>
            break

        hindi_words.append(hin_idx2word.get(predicted_idx, '<UNK>'))
        dec_input = torch.tensor([[predicted_idx]])

    return ' '.join(hindi_words)  # Hint: Combine list into string

print("Translation function ready")

Translation function ready


## 13. Test Translations

* **Evaluate Model:** This cell tests the model on sample sentences from the training set.

**Hints:**
- Loop through specific indices `[0, 5, 10, 20, 30]`
- Compare predicted vs actual translations
- Use ✓ for match, ✗ for mismatch

**What each function does:**
| Function | Description |
|----------|-------------|
| `df['column'][idx]` | Gets value at specified index from DataFrame column |
| `translate(sentence)` | Our custom function - translates English to Hindi |
| `pred == actual` | String comparison - returns True if translations match exactly |

**Documentation:**
- [DataFrame indexing](https://pandas.pydata.org/docs/user_guide/indexing.html) - Access DataFrame values

In [17]:
print("TRANSLATION RESULTS")
print("=" * 70)

for idx in [0, 5, 10, 20, 30]:
    eng = df_small['English'][idx]  # Hint: Column name for English sentences
    actual = df_small['Hindi'][idx]  # Hint: Column name for Hindi sentences
    predicted = translate(eng)  # Hint: Function to translate

    match = "correct" if predicted == actual else "wrong"
    print(f"\n{match} English:   {eng}")
    print(f"  Predicted: {predicted}")
    if predicted != actual:
        print(f"  Actual:    {actual}")

print("\n" + "=" * 70)

TRANSLATION RESULTS

wrong English:   Help!
  Predicted: <UNK>
  Actual:    बचाओ!

wrong English:   Hello!
  Predicted: <UNK>
  Actual:    नमस्कार।

wrong English:   Awesome!
  Predicted: <UNK>
  Actual:    बहुत बढ़िया!

wrong English:   Have fun.
  Predicted: मज़े करना।
  Actual:    मौज करना।

correct English:   Excuse me.
  Predicted: माफ़ कीजिए।



## 14. Try Your Own!

* **Interactive Testing:** This cell lets you test the model on any English sentence.

**Hints:**
- Change the `your_sentence` variable to test different inputs
- Model works best on short, simple sentences similar to training data

**What each function does:**
| Function | Description |
|----------|-------------|
| `translate(sentence)` | Takes English string, returns Hindi translation |
| `print(f"...")` | Formatted string printing with variable interpolation |

**Documentation:**
- [f-strings](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals) - Formatted string literals

In [18]:
# Change this sentence!
your_sentence = "Hey need your help"

translation = translate(your_sentence)  # Hint: Call the translation function
print(f"English: {your_sentence}")
print(f"Hindi:   {translation}")

English: Hey need your help
Hindi:   तुम <UNK> से शादी की।


---

## Summary

**What we built:**
1. **Encoder**: Reads English → creates context
2. **Decoder**: Uses context → generates Hindi

**How it works:**
```
"Hello" → Encoder → [Context Vector] → Decoder → "नमस्ते"
```

**Key Components:**
- Vocabulary: Words ↔ Numbers
- Embeddings: Numbers → Vectors  
- GRU: Process sequences
- Training: Learn from examples

**Congratulations!**  You built a translation system!